In [13]:
import tensorflow as tf
import keras
import os 
import time
#import cv2
import numpy as np
#import matplotlib.pyplot as plt
from keras import optimizers
import keras
import tensorflow as tf
import keras.backend as K
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import load_model
from keras.models import Sequential
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
#import matplotlib.pyplot as plt
#from tqdm import tqdm
from keras.utils import np_utils
#from imgaug import augmenters as iaa    
import itertools
%matplotlib inline
#import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
#from classification_models.keras import Classifiers
# GPU test
from tensorflow.python.client import device_lib
np.random.seed(42)
from keras.models import load_model
# Print version
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.regularizers import l2
import os, sys
import scipy.misc
from glob import glob
import numpy as np
import random 
import shutil 
import keras
import time
import matplotlib.pyplot as plt
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPool2D, Activation,Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import Sequential
from keras.applications.densenet import DenseNet121
from keras.models import load_model
from keras.optimizers import Adam
from keras import optimizers
import pickle
import sys
import cv2
import efficientnet.keras as efn
import tensorflow as tf


np.random.seed(1)

In [14]:
x_train = np.load("x_train_299.npy")
y_train = np.load("y_train_299.npy")
x_train.shape, y_train.shape

((10661, 256, 256, 3), (10661, 2))

In [15]:
x_val = np.load("x_val_299.npy")
y_val = np.load("y_val_299.npy")
x_val.shape, y_val.shape

((1867, 256, 256, 3), (1867, 2))

In [16]:
from tensorflow.python.client import device_lib

In [17]:
print("Keras Version", keras.__version__)
print("Tensorflow Version", tf.__version__)

Keras Version 2.2.4
Tensorflow Version 1.14.0


In [18]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


In [19]:
# Get compute specs
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15752577040242602494,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1410301542
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3674682494408072643
 physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [20]:
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [21]:
def effnet():
    # build model
    base_model = efn.EfficientNetB0(input_shape=(256,256,3), weights='imagenet', include_top=False)
    
    # add GAP layer and 2-ways softmax
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.models.Model(inputs=[base_model.input], outputs=[output])
    model.compile(optimizer='SGD', loss=f1_loss, metrics=['accuracy'] )
    return model


model=None
model=effnet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 32) 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

In [22]:
callbacks = [
    keras.callbacks.ModelCheckpoint('effnetb0.h5', monitor='val_acc', save_best_only=True, mode='max'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.1, verbose=1, patience=5, mode='max')]

In [27]:
model.fit(x=x_train, y=y_train, validation_data = (x_val,y_val), batch_size = 8, epochs=50 ,callbacks=callbacks, verbose=1)

Train on 10661 samples, validate on 1867 samples
Epoch 1/50
10661/10661 [==============================] - 493s 46ms/step - loss: 0.3850 - acc: 0.5000 - val_loss: 0.5106 - val_acc: 0.5000
Epoch 2/50
  376/10661 [>.............................] - ETA: 6:09 - loss: 0.3784 - acc: 0.5000

KeyboardInterrupt: 